In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [13]:
ee.Initialize()

daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
     .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
     .mean()

##Note: there are 7 measurements per day (2555 per year except on leapyears) 

# daymet2 = ee.ImageCollection('NASA/ORNL/DAYMET_V4') \
#      .filter(ee.Filter.date('2000-01-01', '2000-12-31'))

ma_pts = ee.FeatureCollection(
    [
        ee.Feature(ee.Geometry.Point([-70.842918395996, 42.730953216553]), {'site_id': 'LP HM'}),
        ee.Feature(ee.Geometry.Point([-70.842468261719, 42.731742858887]), {'site_id': 'LP LM'})
    ]
)

roi = ee.Geometry.Polygon(
        [
            [
                [-70.8425, 42.7331],
                [-70.8400, 42.7312],
                [-70.8430, 42.7296],
                [-70.8452, 42.7311]
            ]
        ]
    )

dm_precip = daymet.select('prcp')
dm_maxtemp = daymet.select('tmax')
dm_mintemp = daymet.select('tmin')
dm_daylight = daymet.select('dayl')
dm_srad = daymet.select('srad')
dm_snow = daymet.select('swe')
dm_vapor = daymet.select('vp')

prcpVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

daylVis = {
  'min': 0,
  'max': 50000.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

sradVis = {
  'min': 0,
  'max': 400.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

vpVis = {
  'min': 0,
  'max': 1500,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = geemap.Map(center=[40,-100], zoom=4)
Map.addLayer(dm_precip, prcpVis, "Precipitation")
Map.addLayer(dm_maxtemp, prcpVis, "Maxtemp")
Map.addLayer(dm_mintemp, prcpVis, "Mintemp")
Map.addLayer(dm_daylight, daylVis, "Daylight")
Map.addLayer(dm_srad, sradVis, "SRAD")
Map.addLayer(dm_snow, prcpVis, "Snowpack water?")
Map.addLayer(dm_vapor, vpVis, "Vaporpressure")

Map.addLayer(roi, name = "Region of interest")

Map

In [14]:
##Exporting pixel values from points

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = ma_pts,
        scale = 1000 
    )
    return feature

xtract = daymet.map(addDate).map(rasterExtraction).flatten()

final_extracted = geemap.ee_to_pandas(xtract)

display(final_extracted)

out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'ma_extract.csv')
# final_extracted.to_csv(out_csv, index = False)

Exception: Collection query aborted after accumulating over 5000 elements.